В данной задаче у меня представлены два решения за O(N^2^), в которых квадрат достигается при определенных условиях. [Ссылка на решение для ML Blitz 2018](https://github.com/stas-sl/yandex-blitz-ml-2018), в котором он описал ход своих мыслей. Объясняя код, скажу, что мы итерируемся по элементам и удаляем те y, для которых t[i]=t[j], иными словами они лежат на одной прямой по t. Во втором же цикле мы считаем числитель и знаменатель, так как массив отсортирован, то мы можем использовать бинарный поиск, соответственно, все элементы, что лежат слева добавляют 1 (левый бинарный поиск), а все что лежат между левой и правой границей (между левым и правым бинарным поиском) добавляют по 0.5 в числитель.

Объяснение разницы левого и правого бинарного поиска.
Задан отсортированный массив [1,2,2,2,2,3,5,8,9,11], x=2
Правосторонний поиск двойки выдаст в результате 4, в то время как левосторонний выдаст 1 (нумерация с нуля).

Отсюда следует, что количество подряд идущих двоек равно длине отрезка [1;4], то есть 4.

In [ ]:
import bisect

with open('input.txt','r') as f:
    n = int(f.readline())
    t, y = zip(*sorted(list(map(float, f.readline().split())) for i in range(n)))

y_sorted = sorted(y)
num = 0
denom = 0
i = n - 1

while i >= 0:
    j = i
    while j >= 0 and t[i] == t[j]:
        l = bisect.bisect_left(y_sorted, y[j])
        y_sorted.pop(l)
        j -= 1
    for k in range(j + 1, i + 1):
        l = bisect.bisect_left(y_sorted, y[k])
        r = bisect.bisect(y_sorted, y[k])
        num += l + (r - l) / 2
        denom += j + 1
    i = j

with open('output.txt','w') as f:
    f.write(str(num/denom))

Решение через Бинарное дерево. Его проблема в том, что оно не сбалансированное и в худшем случае сложность будет N^2^. Можно попробовать использовать дерево Фенвика, но до него у меня уже у руки не дошли, потому что в итоге было принято решение за N^2^.

In [ ]:
class Node:
    def __init__(self, x):
        self.x = x
        self.left_count = 0
        self.right_count = 0
        self.n = 1
        self.left = None
        self.right = None


class BinaryTree:
    def __init__(self):
        self.root = None

    def add(self, x):
        if self.root is None:
            self.root = Node(x)
        else:
            cur = self.root
            while True:
                if x == cur.x:
                    cur.n += 1
                    break
                elif x < cur.x:
                    cur.left_count += 1
                    if cur.left is None:
                        cur.left = Node(x)
                        break
                    else:
                        cur = cur.left
                else:
                    cur.right_count += 1
                    if cur.right is None:
                        cur.right = Node(x)
                        break
                    else:
                        cur = cur.right

    def count_less_and_eq(self, x):
        less = 0
        eq = 0
        cur = self.root
        while cur:
            if x == cur.x:
                less += cur.left_count
                eq = cur.n
                break
            elif x < cur.x:
                cur = cur.left
            else:
                less += cur.left_count + cur.n
                cur = cur.right
        return less, eq



with open('input.txt', 'r') as file:
    n = int(file.readline())
    arr = []
    for _ in range(n):
        true_val, prev_val = map(float, file.readline().split())
        arr.append((true_val, prev_val))

arr.sort(key=lambda x: x[0])

tree = BinaryTree()
num = 0
denom = 0

i = 0
while i < n:
    j = i
    while j < n and arr[i][0] == arr[j][0]:
        r = tree.count_less_and_eq(arr[j][1])
        num += r[0] + r[1] / 2
        denom += i
        j += 1

    for k in range(i, j):
        tree.add(arr[k][1])

    i = j

print(num / denom)